In [261]:
import sys, os
from pathlib import Path
sys.path.append(str(Path(os.path.abspath('')).absolute().parent.parent.parent))

In [262]:
import random
import numpy as np
import grpc

from shiva.core.communication_objects.helpers_pb2 import Empty
from shiva.core.communication_objects.env_command_pb2 import EnvironmentCommand
from shiva.core.communication_objects.env_step_pb2 import ( EnvStepInput, EnvStepOutput )
from shiva.core.communication_objects.env_specs_pb2 import EnvironmentSpecs
from shiva.core.communication_objects.env_metrics_pb2 import AgentMetrics
from shiva.core.communication_objects.agent_state_pb2 import AgentState
from shiva.core.communication_objects.service_env_pb2_grpc import EnvironmentStub, EnvironmentServicer, add_EnvironmentServicer_to_server

In [263]:
def action2one_hot(action_space: int, action_idx: int, numpy: bool=True) -> np.ndarray:
    '''
        Returns a one-hot encoded numpy.ndarray
    '''
    z = np.zeros(action_space)
    z[action_idx] = 1
    return z if numpy else list(z)

In [264]:
# def get_specs():
#     channel = grpc.insecure_channel('localhost:50051')
#     stub = EnvironmentStub(channel)

#     specs = stub.GetSpecs(Empty())
    
#     return specs

In [265]:
# get_specs()

In [266]:
def get_network_action(action_space):
    return action2one_hot(action_space, random.randint(0, action_space-1), False)

In [289]:
def run():
    channel = grpc.insecure_channel('localhost:50051')
    stub = EnvironmentStub(channel)
    
    specs = stub.GetSpecs(Empty())
    
#     print(specs)
    
    env_in = EnvStepInput()
    
    agent_id = '0'

    action = env_in.agent_actions[agent_id].data.add()
    action.data.extend(get_network_action(specs.action_space.discrete))

    env_in.command = EnvironmentCommand.STEP

    env_state = stub.Step(env_in)
    
#     print(env_state)
    return env_state

In [292]:
for _ in range(1):
    env_output = run()
env_output

agent_states {
  key: "0"
  value {
    data {
      next_observation {
        data: 0.9190667867660522
        data: 0.9888443350791931
        data: 0.3270346522331238
        data: 0.06301821768283844
        data: 0.18770290911197662
      }
      reward: 0.19976165890693665
    }
    data {
      next_observation {
        data: 0.8627603054046631
        data: 0.10960359126329422
        data: 0.005756767001003027
        data: 0.3956153392791748
        data: 0.13649950921535492
      }
      reward: 0.9051660299301147
    }
    data {
      next_observation {
        data: 0.4627743065357208
        data: 0.05693043768405914
        data: 0.10005635023117065
        data: 0.6226266026496887
        data: 0.005160175729542971
      }
      reward: 0.6469001770019531
      done: true
    }
    data {
      next_observation {
        data: 0.9444210529327393
        data: 0.5428276658058167
        data: 0.8832617998123169
        data: 0.37248942255973816
        data: 0.4699847

# Client side

In [91]:
env_in = EnvStepInput()

action = env_in.agent_actions['0'].data.add()
action.data.extend(np.random.rand(5))

action = env_in.agent_actions['0'].data.add()
action.data.extend([np.random.randint(1,5) for _ in range(5)])

env_in.command = EnvironmentCommand.STEP
env_in

agent_actions {
  key: "0"
  value {
    data {
      data: 0.48928579688072205
      data: 0.019176052883267403
      data: 0.6886849403381348
      data: 0.8342480659484863
      data: 0.9397267699241638
    }
    data {
      data: 1.0
      data: 2.0
      data: 4.0
      data: 4.0
      data: 2.0
    }
  }
}

In [ ]:
v = list(env_in.agent_actions.keys())
v

In [ ]:
env_in.agent_actions[v[0]].data

In [ ]:
agent_action = env_in.agent_actions[v[0]].data

In [ ]:
len(agent_action)

In [ ]:
agent_action[0].data

In [ ]:
agent_action[1].data

In [ ]:
all_agents_instances_actions = []

In [ ]:
all_agents_instances_actions.append([ agent_action[i].data for i in range(len(agent_action)) ])

In [ ]:
all_agents_instances_actions[0]

# Server side

In [ ]:
a = AgentState()
a.next_observation.data.extend(np.random.rand(5)) 
a.reward = np.random.rand(1)[0]
a.done = bool(random.randint(0,2))
a

In [271]:
env_state = EnvStepOutput()

a = env_state.agent_states['0'].data.add()
a.next_observation.data.extend(np.random.rand(5))
a.reward = np.random.rand(1)[0]
a.done = bool(random.randint(0, 1))

a = env_state.agent_states['1'].data.add()
a.next_observation.data.extend(np.random.rand(5))
a.reward = np.random.rand(1)[0]
a.done = bool(random.randint(0, 1))

m = env_state.agent_metrics['0'].data.add()
m.steps_per_episode = 1
m.step_count = 2
m.temp_done_counter = 3
m.done_count = 4
m.reward_per_step = 5
m.reward_per_episode = 6
m.reward_total = 7
env_state

agent_states {
  key: "0"
  value {
    data {
      next_observation {
        data: 0.945131778717041
        data: 0.9291877150535583
        data: 0.43990474939346313
        data: 0.7788233160972595
        data: 0.27262768149375916
      }
      reward: 0.6456766128540039
    }
  }
}
agent_states {
  key: "1"
  value {
    data {
      next_observation {
        data: 0.8197154998779297
        data: 0.4252639412879944
        data: 0.07176996022462845
        data: 0.9502240419387817
        data: 0.5882408022880554
      }
      reward: 0.4324740767478943
      done: true
    }
  }
}
agent_metrics {
  key: "0"
  value {
    data {
      steps_per_episode: 1.0
      step_count: 2.0
      temp_done_counter: 3.0
      done_count: 4.0
      reward_per_step: 5.0
      reward_per_episode: 6.0
      reward_total: 7.0
    }
  }
}

In [64]:
env_specs = EnvironmentSpecs()
env_specs.observation_space = 1
env_specs.action_space.discrete = 2
env_specs.action_space.param = 0
env_specs.action_space.acs_space = 2
env_specs.num_instances = 3
env_specs.num_agents_per_instance = 4
env_specs

observation_space: 1
action_space {
  discrete: 2
  acs_space: 2
}
num_instances: 3
num_agents_per_instance: 4

# Test on Parsing different Action shapes

# 1 Agent, 1 Instance, Action Space = 3

In [161]:
act = torch.rand(3)
act

tensor([0.1901, 0.5170, 0.4198])

# 1 Agent, 2 Instances, Action Space = 3

In [177]:
act = torch.rand(2, 3)
act

tensor([[0.0273, 0.8652, 0.6796],
        [0.8758, 0.5200, 0.5812]])

In [178]:
act.shape

torch.Size([2, 3])

In [179]:
len(act.shape) == 2

True

In [180]:
print('Agent 0')
for n_instance in range(act.shape[0]):
    print('\tInstance {} --> {}'.format(n_instance, act[n_instance]))

Agent 0
	Instance 0 --> tensor([0.0273, 0.8652, 0.6796])
	Instance 1 --> tensor([0.8758, 0.5200, 0.5812])


# 3 Agents, 2 Instances, Action Space = 3

In [166]:
act = torch.rand(3, 2, 3)
act

tensor([[[0.0319, 0.6395, 0.4325],
         [0.0143, 0.0948, 0.4598]],

        [[0.3606, 0.9219, 0.6354],
         [0.3092, 0.5361, 0.5611]],

        [[0.6553, 0.2330, 0.5499],
         [0.7051, 0.2312, 0.9553]]])

In [167]:
act.shape

torch.Size([3, 2, 3])

In [168]:
len(act.shape) == 3

True

In [170]:
for n_agent in range(act.shape[0]):
    print('Agent {}'.format(n_agent))
    for n_instance in range(act.shape[1]):
        print('\tInstance {} --> {}'.format(n_instance, act[n_agent, n_instance]))

Agent 0
	Instance 0 --> tensor([0.0319, 0.6395, 0.4325])
	Instance 1 --> tensor([0.0143, 0.0948, 0.4598])
Agent 1
	Instance 0 --> tensor([0.3606, 0.9219, 0.6354])
	Instance 1 --> tensor([0.3092, 0.5361, 0.5611])
Agent 2
	Instance 0 --> tensor([0.6553, 0.2330, 0.5499])
	Instance 1 --> tensor([0.7051, 0.2312, 0.9553])


# Test for parsing the EnvOutputStep

- MultiAgent
- MultiInstance

In [327]:
env_output = run()

In [328]:
env_output

agent_states {
  key: "0"
  value {
    data {
      next_observation {
        data: 0.9263609051704407
        data: 0.6020965576171875
        data: 0.8852146863937378
        data: 0.02316807396709919
        data: 0.45889753103256226
      }
      reward: 0.7580342888832092
      done: true
    }
  }
}
agent_metrics {
  key: "0"
  value {
    data {
      steps_per_episode: 1.0
      step_count: 2.0
      temp_done_counter: 3.0
      done_count: 4.0
      reward_per_step: 5.0
      reward_per_episode: 6.0
      reward_total: 7.0
    }
  }
}

In [360]:
traj = {}
for a_id in list(env_output.agent_states.keys()):
    traj[a_id] = []
    for state in list(env_output.agent_states[a_id].data):
        experience = [state.next_observation.data, state.reward, state.done]
        traj[a_id].append(experience)

In [374]:
traj

{'0': [[[0.9263609051704407, 0.6020965576171875, 0.8852146863937378, 0.02316807396709919, 0.45889753103256226],
   0.7580342888832092,
   True]]}

In [375]:
def from_EnvStepOutput_to_standard(env_output):
    agent_ids = list(env_output.agent_states.keys())
    agent_id = agent_ids[0] # SINGLE AGENT PROCESSING
    traj = {}
    for a_id in agent_ids:
        traj[a_id] = []
        for state in list(env_output.agent_states[a_id].data):
            experience = [state.next_observation.data, state.reward, state.done]
            traj[a_id].append(experience)
    return traj['0'][0]

In [376]:
obs, rew, done = from_EnvStepOutput_to_standard(env_output)

In [377]:
obs, rew, done

([0.9263609051704407, 0.6020965576171875, 0.8852146863937378, 0.02316807396709919, 0.45889753103256226],
 0.7580342888832092,
 True)

In [418]:
def set_metrics(self):
    agent_ids = list(env_output.agent_metrics.keys())
    agent_id = agent_ids[0] # SINGLE AGENT PROCESSING
    metric = {}
    for a_id in agent_ids:
        metric[a_id] = env_output.agent_metrics[a_id].data[0]
    return metric

In [419]:
m = set_metrics(env_output)